In [1]:
import pandas as pd
from openai import OpenAI
import os
from dotenv import load_dotenv
from tqdm import tqdm
import time

In [2]:
# Load environment variables from the .env file
load_dotenv(".env")

True

In [3]:
df = pd.read_csv("data/naturals_reviews.csv")

In [4]:
len(df)

53277

In [5]:
df.head()

,id_review,caption,relative_date,review_date,retrieval_date,rating,username,n_review_user,place_id
0,ChZDSUhNMG9nS0VJQ0FnSUR2dmVPaVR3EAE,NaN,an hour ago,2024-12-23 15:22:09.506774,2024-12-23 15:22:09.506868,5.0,Vasanthu,0,ChIJB0quoslnUjoRf_vm8BiGHsM
1,ChZDSUhNMG9nS0VJQ0FnSUR2NVptekJnEAE,"Rosika did an excellent facial, great neck mas...",22 hours ago,2024-12-23 15:22:09.507031,2024-12-23 15:22:09.507073,5.0,Sujatha Madangopal,0,ChIJB0quoslnUjoRf_vm8BiGHsM
2,ChdDSUhNMG9nS0VJQ0FnSUR2NGJMZndBRRAB,had a great hair washing and blow dry experien...,2 days ago,2024-12-21 15:22:09.507740,2024-12-23 15:22:09.507785,5.0,080 harshitha sridhar,0,ChIJB0quoslnUjoRf_vm8BiGHsM
3,ChdDSUhNMG9nS0VJQ0FnSUR2dHN1WG9nRRAB,Excellent face massage by Goweri Thankyou you...,2 days ago,2024-12-21 15:22:09.507917,2024-12-23 15:22:09.507958,5.0,Varun Bala Vk,0,ChIJB0quoslnUjoRf_vm8BiGHsM
4,ChZDSUhNMG9nS0VJQ0FnSUNLdUtTR1NREAE,Aakash raj - Did very good haircut and shave s...,3 days ago,2024-12-20 15:22:09.508090,2024-12-23 15:22:09.508131,5.0,Rajesh Gopalsamy,23,ChIJB0quoslnUjoRf_vm8BiGHsM


In [6]:
df["place_id"].value_counts()

place_id
ChIJp4gXqlZaUjoR0qOHmTRnuOg    1393
ChIJdZGLpRdmUjoRpfmePJPrL_I    1301
ChIJdZIG_wVnUjoRPMLG0KojMZ4    1233
ChIJ6aCWRmJnUjoRRmaT-wCYdLE    1207
ChIJXS3RwKpfUjoReizoYiuHWeQ    1109
                               ... 
ChIJc2rZU8NZUjoRpHxFHkKBqzw       4
ChIJhbOICDdeUjoRTKDdJNSdE-U       4
ChIJZfEOCVOIUjoRLDvAs5q9EJ0       2
ChIJaQiH1DJhUjoRlxzzTRuXcJU       1
ChIJI2180bJfUjoRQ4DFD7PjE8c       1
Name: count, Length: 146, dtype: int64

In [7]:
len(df["place_id"].value_counts())

146

In [8]:
df["relative_date"].value_counts()

relative_date
4 months ago      7052
2 months ago      5164
3 months ago      5140
9 months ago      5071
6 months ago      5039
5 months ago      4861
8 months ago      4820
10 months ago     4660
7 months ago      4122
11 months ago     3787
2 weeks ago       1242
3 weeks ago        876
4 weeks ago        428
2 days ago         256
3 days ago         171
4 days ago         150
6 days ago         127
5 days ago         117
22 hours ago        13
23 hours ago        13
21 hours ago        12
16 hours ago        12
13 hours ago        11
12 hours ago        11
17 hours ago        10
11 hours ago        10
9 hours ago          9
10 hours ago         9
18 hours ago         9
7 hours ago          9
15 hours ago         8
6 hours ago          7
14 hours ago         7
2 hours ago          6
3 hours ago          6
4 hours ago          6
20 hours ago         6
8 hours ago          5
19 hours ago         3
5 hours ago          3
an hour ago          2
20 minutes ago       1
15 minutes ago      

In [9]:
df["review_date"].value_counts()

review_date
2024-12-23 15:22:09.506774    1
2024-08-24 06:45:58.700052    1
2024-02-24 07:00:13.192234    1
2024-02-24 07:00:13.192426    1
2024-02-24 07:00:13.192615    1
                             ..
2024-10-23 23:25:40.378043    1
2024-10-23 23:25:40.378322    1
2024-10-23 23:25:40.378543    1
2024-10-23 23:25:40.378744    1
2024-01-24 14:20:28.454014    1
Name: count, Length: 53277, dtype: int64

In [10]:
df['caption'].isna().sum()

6755

In [11]:
df = df[df['caption'].notna()]

In [12]:
len(df)

46522

In [13]:
df.head()

,id_review,caption,relative_date,review_date,retrieval_date,rating,username,n_review_user,place_id
1,ChZDSUhNMG9nS0VJQ0FnSUR2NVptekJnEAE,"Rosika did an excellent facial, great neck mas...",22 hours ago,2024-12-23 15:22:09.507031,2024-12-23 15:22:09.507073,5.0,Sujatha Madangopal,0,ChIJB0quoslnUjoRf_vm8BiGHsM
2,ChdDSUhNMG9nS0VJQ0FnSUR2NGJMZndBRRAB,had a great hair washing and blow dry experien...,2 days ago,2024-12-21 15:22:09.507740,2024-12-23 15:22:09.507785,5.0,080 harshitha sridhar,0,ChIJB0quoslnUjoRf_vm8BiGHsM
3,ChdDSUhNMG9nS0VJQ0FnSUR2dHN1WG9nRRAB,Excellent face massage by Goweri Thankyou you...,2 days ago,2024-12-21 15:22:09.507917,2024-12-23 15:22:09.507958,5.0,Varun Bala Vk,0,ChIJB0quoslnUjoRf_vm8BiGHsM
4,ChZDSUhNMG9nS0VJQ0FnSUNLdUtTR1NREAE,Aakash raj - Did very good haircut and shave s...,3 days ago,2024-12-20 15:22:09.508090,2024-12-23 15:22:09.508131,5.0,Rajesh Gopalsamy,23,ChIJB0quoslnUjoRf_vm8BiGHsM
5,ChZDSUhNMG9nS0VJQ0FnSUR2M01DbWR3EAE,Excellent hair spa service by Rosika,4 days ago,2024-12-19 15:22:09.508263,2024-12-23 15:22:09.508304,5.0,Kaviya,19,ChIJB0quoslnUjoRf_vm8BiGHsM


In [14]:
reviews = df["caption"]

In [15]:
len(reviews)

46522

In [16]:
reviews

1        Rosika did an excellent facial, great neck mas...
2        had a great hair washing and blow dry experien...
3        Excellent face massage by Goweri  Thankyou you...
4        Aakash raj - Did very good haircut and shave s...
5                     Excellent hair spa service by Rosika
                               ...                        
53272    Anju service is very good and neat with patience.
53273    Great haircut and service by Mr.Saravanan . Wi...
53274    Fantastic service, very happy with the staff. ...
53275    Saravanan done will haircut and style for me 🔥🔥 …
53276                 Very nice salon with skillful barber
Name: caption, Length: 46522, dtype: object

In [17]:
# Function to analyze sentiment
def analyze_sentiment(text):
    """
    Analyze the sentiment of a given text using OpenAI's GPT models.

    Args:
        text (str): The text to analyze.

    Returns:
        str: The sentiment analysis result (e.g., "Positive", "Negative", "Neutral").
    """
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    
    try:
        response = client.chat.completions.create(
            model="gpt-4",  # Replace with the model you prefer
            messages=[
                {"role": "system", "content": "You are a sentiment analysis assistant."},
                {"role": "user", "content": f"Please classify the sentiment of the following text as Positive, Negative, or Neutral:\n\n{text}"}
            ],
            temperature=0.0
        )

        sentiment = response.choices[0].message.content
        return sentiment
        
    except Exception as e:
        print(f"Error during sentiment analysis: {e}")
        return None

In [18]:
response = analyze_sentiment(reviews[1])

In [19]:
response

'Positive'

In [20]:
reviews[1]

'Rosika did an excellent facial, great neck massage,, hair colour and great job by kala - waxing..'

In [21]:
tqdm.pandas()

In [22]:
def analyze_sentiment_batch(texts):
    """
    Analyze sentiment for a batch of texts using OpenAI API.

    Args:
        texts (list of str): List of texts to analyze.

    Returns:
        list of str: List of sentiment analysis results.
    """
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    
    try:
        # Batch prompt with numbered texts for clarity
        batch_prompt = "\n".join(
            [f"Text {i+1}: {text}" for i, text in enumerate(texts)]
        )
        messages = [
            {"role": "system", "content": "You are a sentiment analysis assistant."},
            {"role": "user", "content": f"Classify the sentiment of the following texts only as either Positive, Negative, Mixed, or Neutral:\n\n{batch_prompt}." 
                    f"If you find text in Tamil, translate first and then classify."
                    f"If you find spelling or grammatical errors, correct the spelling and/or grammar and then classify." }
        ]
        
        response = client.chat.completions.create(
            model="gpt-4",  # Use "gpt-3.5-turbo" for faster response if accuracy is acceptable
            messages=messages,
            temperature=0.0
        )
        
        # Parse response into individual results
        results = response.choices[0].message.content.split("\n")
        return [result.split(":")[1].strip() for result in results]
    
    except Exception as e:
        print(f"Error during sentiment analysis: {e}")
        return [None] * len(texts)

In [23]:
len(df)

46522

In [24]:
df = df.dropna(subset="caption")

In [25]:
len(df)

46522

In [26]:
sample_df = df

In [27]:
sample_df

,id_review,caption,relative_date,review_date,retrieval_date,rating,username,n_review_user,place_id
1,ChZDSUhNMG9nS0VJQ0FnSUR2NVptekJnEAE,"Rosika did an excellent facial, great neck mas...",22 hours ago,2024-12-23 15:22:09.507031,2024-12-23 15:22:09.507073,5.0,Sujatha Madangopal,0,ChIJB0quoslnUjoRf_vm8BiGHsM
2,ChdDSUhNMG9nS0VJQ0FnSUR2NGJMZndBRRAB,had a great hair washing and blow dry experien...,2 days ago,2024-12-21 15:22:09.507740,2024-12-23 15:22:09.507785,5.0,080 harshitha sridhar,0,ChIJB0quoslnUjoRf_vm8BiGHsM
3,ChdDSUhNMG9nS0VJQ0FnSUR2dHN1WG9nRRAB,Excellent face massage by Goweri Thankyou you...,2 days ago,2024-12-21 15:22:09.507917,2024-12-23 15:22:09.507958,5.0,Varun Bala Vk,0,ChIJB0quoslnUjoRf_vm8BiGHsM
4,ChZDSUhNMG9nS0VJQ0FnSUNLdUtTR1NREAE,Aakash raj - Did very good haircut and shave s...,3 days ago,2024-12-20 15:22:09.508090,2024-12-23 15:22:09.508131,5.0,Rajesh Gopalsamy,23,ChIJB0quoslnUjoRf_vm8BiGHsM
5,ChZDSUhNMG9nS0VJQ0FnSUR2M01DbWR3EAE,Excellent hair spa service by Rosika,4 days ago,2024-12-19 15:22:09.508263,2024-12-23 15:22:09.508304,5.0,Kaviya,19,ChIJB0quoslnUjoRf_vm8BiGHsM
...,...,...,...,...,...,...,...,...,...
53272,ChZDSUhNMG9nS0VJQ0FnSUROcVB1ZVFBEAE,Anju service is very good and neat with patience.,11 months ago,2024-01-24 14:20:24.224514,2024-12-24 14:20:24.224568,5.0,sundari Nana,0,ChIJUUOJu_tmUjoRL7BsVmdNZnw
53273,ChdDSUhNMG9nS0VJQ0FnSUROOEotVnZBRRAB,Great haircut and service by Mr.Saravanan . Wi...,11 months ago,2024-01-24 14:20:28.453337,2024-12-24 14:20:28.453422,5.0,Keshav Balaji babu,0,ChIJUUOJu_tmUjoRL7BsVmdNZnw
53274,ChZDSUhNMG9nS0VJQ0FnSUMxczZPUmRnEAE,"Fantastic service, very happy with the staff. ...",11 months ago,2024-01-24 14:20:28.453608,2024-12-24 14:20:28.453665,5.0,Amrutha Gopinath,0,ChIJUUOJu_tmUjoRL7BsVmdNZnw
53275,ChdDSUhNMG9nS0VJQ0FnSUMxMGRLdWlnRRAB,Saravanan done will haircut and style for me 🔥🔥 …,11 months ago,2024-01-24 14:20:28.453813,2024-12-24 14:20:28.453872,5.0,ghatam sarvesh,0,ChIJUUOJu_tmUjoRL7BsVmdNZnw


In [28]:
tqdm.pandas()

batch_size = 5

sample_df['sentiment'] = None

for i in tqdm(range(0, len(sample_df), batch_size), desc="Processing Batches"):
    batch = sample_df['caption'].iloc[i:i+batch_size]
    sentiments = analyze_sentiment_batch(batch.tolist())
    sample_df.loc[batch.index, 'sentiment'] = sentiments

Processing Batches: 100%|█████████████████████████████████████████| 9305/9305 [5:40:31<00:00,  2.20s/it]


In [29]:
sample_df.head()

,id_review,caption,relative_date,review_date,retrieval_date,rating,username,n_review_user,place_id,sentiment
1,ChZDSUhNMG9nS0VJQ0FnSUR2NVptekJnEAE,"Rosika did an excellent facial, great neck mas...",22 hours ago,2024-12-23 15:22:09.507031,2024-12-23 15:22:09.507073,5.0,Sujatha Madangopal,0,ChIJB0quoslnUjoRf_vm8BiGHsM,Positive
2,ChdDSUhNMG9nS0VJQ0FnSUR2NGJMZndBRRAB,had a great hair washing and blow dry experien...,2 days ago,2024-12-21 15:22:09.507740,2024-12-23 15:22:09.507785,5.0,080 harshitha sridhar,0,ChIJB0quoslnUjoRf_vm8BiGHsM,Positive
3,ChdDSUhNMG9nS0VJQ0FnSUR2dHN1WG9nRRAB,Excellent face massage by Goweri Thankyou you...,2 days ago,2024-12-21 15:22:09.507917,2024-12-23 15:22:09.507958,5.0,Varun Bala Vk,0,ChIJB0quoslnUjoRf_vm8BiGHsM,Positive
4,ChZDSUhNMG9nS0VJQ0FnSUNLdUtTR1NREAE,Aakash raj - Did very good haircut and shave s...,3 days ago,2024-12-20 15:22:09.508090,2024-12-23 15:22:09.508131,5.0,Rajesh Gopalsamy,23,ChIJB0quoslnUjoRf_vm8BiGHsM,Positive
5,ChZDSUhNMG9nS0VJQ0FnSUR2M01DbWR3EAE,Excellent hair spa service by Rosika,4 days ago,2024-12-19 15:22:09.508263,2024-12-23 15:22:09.508304,5.0,Kaviya,19,ChIJB0quoslnUjoRf_vm8BiGHsM,Positive


In [30]:
sample_df.to_csv("data/naturals_sentiments.csv")

In [31]:
len(sample_df)

46522

### Read Data

In [32]:
sentiments_df = pd.read_csv("data/naturals_sentiments.csv")

In [33]:
sentiments_df.head()

,Unnamed: 0,id_review,caption,relative_date,review_date,retrieval_date,rating,username,n_review_user,place_id,sentiment
0,1,ChZDSUhNMG9nS0VJQ0FnSUR2NVptekJnEAE,"Rosika did an excellent facial, great neck mas...",22 hours ago,2024-12-23 15:22:09.507031,2024-12-23 15:22:09.507073,5.0,Sujatha Madangopal,0,ChIJB0quoslnUjoRf_vm8BiGHsM,Positive
1,2,ChdDSUhNMG9nS0VJQ0FnSUR2NGJMZndBRRAB,had a great hair washing and blow dry experien...,2 days ago,2024-12-21 15:22:09.507740,2024-12-23 15:22:09.507785,5.0,080 harshitha sridhar,0,ChIJB0quoslnUjoRf_vm8BiGHsM,Positive
2,3,ChdDSUhNMG9nS0VJQ0FnSUR2dHN1WG9nRRAB,Excellent face massage by Goweri Thankyou you...,2 days ago,2024-12-21 15:22:09.507917,2024-12-23 15:22:09.507958,5.0,Varun Bala Vk,0,ChIJB0quoslnUjoRf_vm8BiGHsM,Positive
3,4,ChZDSUhNMG9nS0VJQ0FnSUNLdUtTR1NREAE,Aakash raj - Did very good haircut and shave s...,3 days ago,2024-12-20 15:22:09.508090,2024-12-23 15:22:09.508131,5.0,Rajesh Gopalsamy,23,ChIJB0quoslnUjoRf_vm8BiGHsM,Positive
4,5,ChZDSUhNMG9nS0VJQ0FnSUR2M01DbWR3EAE,Excellent hair spa service by Rosika,4 days ago,2024-12-19 15:22:09.508263,2024-12-23 15:22:09.508304,5.0,Kaviya,19,ChIJB0quoslnUjoRf_vm8BiGHsM,Positive


In [34]:
sentiments_df["sentiment"].value_counts()

sentiment
Positive                                                                                                       43866
Neutral                                                                                                         1220
Negative                                                                                                        1010
Mixed                                                                                                            294
Neutral (The sentiment is unclear due to missing information)                                                      2
                                                                                                               ...  
Neutral (The text "Pramila" is a name and does not express any sentiment)                                          1
Positive (The text "Bk hairstyle super service" expresses a positive sentiment towards a hairstyle service)        1
Neutral (The text "Gowsi" is a name and does not expre

In [35]:
# Get value counts of the 'sentiment' column
value_counts = sentiments_df["sentiment"].value_counts()

sentiments_df[sentiments_df["sentiment"].isin(value_counts[value_counts <= 2].index)]


,Unnamed: 0,id_review,caption,relative_date,review_date,retrieval_date,rating,username,n_review_user,place_id,sentiment
2375,2942,ChZDSUhNMG9nS0VJQ0FnSUR6aHN2WWV3EAE,I am statifed the hair services,6 months ago,2024-06-23 16:33:29.745855,2024-12-23 16:33:29.745903,5.0,Jee Vithya,0,ChIJ38JlxW1lUjoRVB-uo0b6CZQ,"Positive (Assuming ""statifed"" is a typo for ""s..."
4725,5572,ChdDSUhNMG9nS0VJQ0FnSURQbExDT21nRRAB,Hidrafacial nalla result kedachathu Beulah,3 weeks ago,2024-12-02 18:08:07.655470,2024-12-23 18:08:07.655511,4.0,Roja papitha,0,ChIJX2cOPjxnUjoRehfttXmstSA,Beulah's HydraFacial gave good results. - Posi...
4726,5574,ChZDSUhNMG9nS0VJQ0FnSUNQc2ZiakN3EAE,It's very good work Beulah ♥️,3 weeks ago,2024-12-02 18:08:07.655850,2024-12-23 18:08:07.655893,5.0,Arthipriya R,1,ChIJX2cOPjxnUjoRehfttXmstSA,It's very good work Beulah ♥️ - Positive
4727,5575,ChdDSUhNMG9nS0VJQ0FnSURua192V3VnRRAB,Had a hair cut which was fabulous ....great wo...,2 months ago,2024-10-23 18:08:07.656043,2024-12-23 18:08:07.656129,5.0,Bhoomi Mishra,0,ChIJX2cOPjxnUjoRehfttXmstSA,Had a hair cut which was fabulous ....great wo...
4728,5577,ChdDSUhNMG9nS0VJQ0FnSUNuMF9UanNBRRAB,I had a wonderful Hydra facial at *Naturals Sa...,2 months ago,2024-10-23 18:08:11.797140,2024-12-23 18:08:11.797206,5.0,Anitha Anitha,0,ChIJX2cOPjxnUjoRehfttXmstSA,I had a wonderful Hydra facial at *Naturals Sa...
...,...,...,...,...,...,...,...,...,...,...,...
45065,51539,ChdDSUhNMG9nS0VJQ0FnSUNYMDhMQzJRRRAB,I done facial..So much Satisfied.. Thank you m...,2 months ago,2024-10-24 13:41:58.731547,2024-12-24 13:41:58.731608,5.0,Bhuvanesh Kumar,0,ChIJu08uvNxvUjoRnU8vgmdp4qc,I had a facial. So much satisfaction. Thank yo...
45066,51540,ChZDSUhNMG9nS0VJQ0FnSUNYclktUGV3EAE,Latha this person did really well,2 months ago,2024-10-24 13:41:58.731767,2024-12-24 13:41:58.731820,5.0,Seneka Ravi,0,ChIJu08uvNxvUjoRnU8vgmdp4qc,"Latha, this person did really well. - Positive"
45067,51541,ChdDSUhNMG9nS0VJQ0FnSUNYclpYdm5BRRAB,Latha and Raji Serrvice was very good,2 months ago,2024-10-24 13:41:58.731974,2024-12-24 13:41:58.732026,5.0,Meena RM,0,ChIJu08uvNxvUjoRnU8vgmdp4qc,Latha and Raji's service was very good. - Posi...
45068,51542,ChZDSUhNMG9nS0VJQ0FnSUN4cjhfdGZBEAE,Wonderful service from Rajesh. Please maintain...,2 months ago,2024-10-24 13:41:58.732160,2024-12-24 13:41:58.732210,5.0,Manoj Karan,4,ChIJu08uvNxvUjoRnU8vgmdp4qc,Wonderful service from Rajesh. Please maintain...
